# Building AI Assistant

In [1]:
#. import modules 
import os 
import gradio as gr 
from dotenv import load_dotenv 
from openai import OpenAI 

In [2]:
# load environment variables 
load_dotenv(override=True)

True

In [3]:
# get the keys 
api_key=os.getenv("OPENAI_API_KEY")
gemini_api_key=os.getenv("GOOGLE_API_KEY")
ollama_api_key=os.getenv("OLLAMA_API_KEY")

# get the base url for gemini and ollama 
ollama_base_url=os.getenv("OLLAMA_BASE_URL")
gemini_base_url=os.getenv("GEMINI_BASE_URL")

In [4]:
# create model instances 
openai=OpenAI()
gemini=OpenAI(base_url=gemini_base_url, api_key=gemini_api_key)
ollama=OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)

In [5]:
# We will change the system prompt later on 
system_message="You are a helpful assistant"

# Creating callbacks 
The main purpose of this function will be to take the prior conversation and return to response

In [6]:
def chat(message, history, model_name="mistral", model:OpenAI=ollama):
    history=[{"role": h["role"], "content": h["content"]} for h in history]

    # build the message with system message + history + user prompt 
    messages=[
        {"role": "system", "content": system_message} 
    ] + history + [
        {"role": "user", "content": message}
    ]

    response=model.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [7]:
# instead of calling interface - call ChatInterface to get chat looking interface for user 
model_selector = gr.Dropdown(["mistral", "llama3.1", "phi4-mini", "gemma3"], label="Select model", value="mistral")

gr.ChatInterface(fn=chat, additional_inputs=[model_selector], type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [9]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [10]:
def chat(message, history, model_name="mistral", model:OpenAI=ollama):

    # get the history 
    history = [{"role":h["role"], "content":h["content"]} for h in history]

    # create new variable relevant system message 
    relevant_system_message = system_message
    if "belt" in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    elif "shoes" in message.lower():
        relevant_system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = model.chat.completions.create(model=model_name, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [11]:
# instead of calling interface - call ChatInterface to get chat looking interface for user 
model_selector = gr.Dropdown(["mistral", "llama3.1", "phi4-mini", "gemma3"], label="Select model", value="mistral")

gr.ChatInterface(fn=chat, additional_inputs=[model_selector], type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
